In [17]:
import tensorflow as tf

(trainX, trainY), (validX, validY) = tf.keras.datasets.fashion_mnist.load_data()

trainX[0]
trainX.shape
validX.shape
# 정답 레이블, 사진의 종류 10가지
trainY

class_names=['T-shirt/top', 'Trouser', 'Pullover', 'Dress','Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [18]:
import numpy as np

trainX = trainX / 255.0
validX = validX / 255.0

trainX = trainX.reshape((trainX.shape[0], trainX.shape[1], trainX.shape[2], 1))
validX = validX.reshape((validX.shape[0], validX.shape[1], validX.shape[2], 1))

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])



In [20]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 10,
    zoom_range = 0.1,
    width_shift_range = 0.1, 
    height_shift_range = 0.1, 
    horizontal_flip = True, 
    vertical_flip = False,
)

batch_size = 32

trainD = datagen.flow(trainX, trainY, batch_size=batch_size)
validD = datagen.flow(validX, validY, batch_size=batch_size)

print(img_iter)
datagen.fit(trainX)


# gen = ImageDataGenerator(width_shift_range=shift_fraction,
#                          height_shift_range=shift_fraction,
#                          horizontal_flip=True)
# batches = gen.flow(X_train, y_train, batch_size=batch_size)
# val_batches = gen.flow(X_val, y_val, batch_size=batch_size)

# fashion_train=fashion_model.fit_generator(batches, steps_per_epoch=X_train.shape[0]//batch_size, 
#                                           epochs=epochs,validation_data=val_batches, 
#                                                    validation_steps=X_val.shape[0]//batch_size, use_multiprocessing=True)

model.summary()

In [22]:


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# model.fit(trainX, trainY, validation_data=(validX, validY), epochs=5)
model.fit_generator(trainD, steps_per_epoch=len(trainX)//batch_size, 
                    validation_data=(validD),validation_steps=validX.shape[0]//batch_size, epochs=5)



C:\Users\sunny\AppData\Local\Temp\ipykernel_16188\2267164362.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(trainD, steps_per_epoch=len(trainX)//batch_size,


Epoch 1/5
1875/1875 [==============================] - 68s 35ms/step - loss: 0.5278 - accuracy: 0.8048 - val_loss: 0.5255 - val_accuracy: 0.8111
Epoch 2/5
1875/1875 [==============================] - 67s 36ms/step - loss: 0.4722 - accuracy: 0.8239 - val_loss: 0.4718 - val_accuracy: 0.8255
Epoch 3/5
1875/1875 [==============================] - 60s 32ms/step - loss: 0.4457 - accuracy: 0.8339 - val_loss: 0.4583 - val_accuracy: 0.8272
Epoch 4/5
1875/1875 [==============================] - 34s 18ms/step - loss: 0.4226 - accuracy: 0.8430 - val_loss: 0.4557 - val_accuracy: 0.8274
Epoch 5/5
1875/1875 [==============================] - 37s 20ms/step - loss: 0.4081 - accuracy: 0.8470 - val_loss: 0.4292 - val_accuracy: 0.8382


In [25]:
#!/usr/bin/python

#Category 2 - Iris 꽃 분류
#
#Fully Connected Layer (Dense)를 활용한 분류 모델 (Classification)
#tensorflow-datasets 를 활용한 데이터 전처리

#For this task you will train a classifier for Iris flowers using the Iris dataset
#The final layer in your neural network should look like: tf.keras.layers.
#Dense(3, activation=tf.nn.softmax)
#The input layer will expect data in the shape (4,)
#We've given you some starter code for preprocessing the data
#You'll need to implement the preprocess function for data.map

#이 작업에서는 Iris 데이터 세트를 사용하여 Iris 꽃 분류기를 훈련시킵니다.
#신경망의 마지막 계층은 tf.keras.layers와 같아야합니다.
#Dense(3, activation='softmax')
#입력 레이어는 모양의 데이터를 기대합니다 (4,)
#데이터 전처리를위한 스타터 코드를 제공했습니다
#data.map에 대한 전처리 기능을 구현해야합니다.

import numpy as np
import tensorflow as tf
import tensorflow.keras.datasets as tfds

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint


def preprocess(data):
	# 코드를 입력하세요
	x = data['features']
	y = data['label']
	y = tf.one_hot(y, 3)
	return x, y

train_dataset = tfds.load('iris', split='train[:80%]')
valid_dataset = tfds.load('iris', split='train[80%:]')


batch_size=10
train_data = train_dataset.map(preprocess).batch(batch_size)
valid_data = valid_dataset.map(preprocess).batch(batch_size)


model = tf.keras.models.Sequential([
	# input_shape는 X의 feature 갯수가 4개 이므로 (4, )로 지정합니다.
	Dense(512, activation='relu', input_shape=(4,)),
	Dense(256, activation='relu'),
	Dense(128, activation='relu'),
	Dense(64, activation='relu'),
	Dense(32, activation='relu'),
	# Classification을 위한 Softmax, 클래스 갯수 = 3개
	Dense(3, activation='softmax'),
])

model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

checkpoint_path = "my_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
	save_weights_only=True, 
	ave_best_only=True, 
	monitor='val_loss', 
	erbose=1
)

history = model.fit(train_data,
	validation_data=(valid_data),
	epochs=20,
	callbacks=[checkpoint], 
)


# checkpoint 를 저장한 파일명을 입력합니다.
model.load_weights(checkpoint_path)


AttributeError: module 'tensorflow.keras.datasets' has no attribute 'load'